In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from scipy.spatial.distance import squareform, pdist
import os
import geopy
from geopy import distance

In [6]:
 # adjacency matrix
landLocation = pd.read_csv(
    "../disneyenv/disneyenv/envs/data/landLocation.csv")
walking_speed = 0.0804672  # km/min

np.max(squareform(pdist(landLocation[[
                                   "longitude", "latitude"]], lambda u, v: geopy.distance.geodesic(u, v).km/walking_speed)))

19.59417004116794

In [8]:
df = pd.read_csv("../disneyenv/disneyenv/envs/data/disneyRideTimes.csv")

/tmp/ipykernel_110299/2545870254.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../disneyenv/disneyenv/envs/data/disneyRideTimes.csv")


In [116]:
df2 = pd.read_csv("../disneyenv/disneyenv/envs/data/hourlyWeather.csv")

In [13]:
ridesinfo = pd.read_csv(
            "../disneyenv/disneyenv/envs/data/rideDuration.csv")
rides = ridesinfo["id"].unique()

In [119]:
df2["dateTime"] = pd.to_datetime(df2["dateTime"])
df2["date"] = df2["dateTime"].dt.date
df2 = df2.set_index("dateTime").sort_index()

In [9]:
df["dateTime"] = pd.to_datetime(df["dateTime"])
df["date"] = df["dateTime"].dt.date
df = df.set_index(["rideID", "dateTime"]).sort_index()

In [68]:
sample_ts = datetime.strptime("2018-03-11 08:41:21.330527", "%Y-%m-%d %H:%M:%S.%f")
target_df = df[df["date"] == sample_ts.date()]
input_waitTime_index = pd.MultiIndex.from_arrays(
            [np.array(rides), np.repeat(sample_ts, len(rides))])

In [69]:
target_ilocs = target_df.index.get_indexer(input_waitTime_index, method="ffill")
valid_targets = np.where(target_ilocs != -1)[0]

In [70]:
valid_targets = valid_targets[
                target_df.index[target_ilocs[valid_targets]].get_level_values(
                    0) == input_waitTime_index[valid_targets].get_level_values(0)]

In [71]:
waitTime = np.repeat(np.nan, len(input_waitTime_index))

In [72]:
waitTime[valid_targets] = target_df.iloc[target_ilocs[valid_targets]
                                                     ].waitMins.values

In [73]:
waitTime = np.nan_to_num(waitTime)

In [74]:
waitTime

array([ 0.,  5.,  0., 15.,  5.,  0.,  0.,  0.,  0.,  5.,  5.,  5.,  0.,
        0., 10.,  5.,  0.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,
        0.,  5.,  0.,  5.,  0., 10.,  0., 30.,  5.,  0.,  5.,  5.,  0.,
        5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10., 15.,
        0.,  0.,  0.,  5.,  5.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  5.,  5.,  5.,  0.,  0.,  0., 10.,  5.,  0.,
        0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  0., 15.,  0.,  0.,  0.,
        0.,  0.])

In [75]:
operationStatus = np.repeat(0, len(input_waitTime_index))
operationStatus[valid_targets] = (
    target_df.iloc[target_ilocs[valid_targets]].status == "Operating")

In [76]:
operationStatus

array([1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [35]:
target_df.index[valid_targets]

MultiIndex([(353289, '2018-07-24 00:00:14'),
            (353289, '2018-07-24 00:30:05'),
            (353289, '2018-07-24 01:00:15'),
            (353289, '2018-07-24 01:30:04'),
            (353289, '2018-07-24 02:00:14'),
            (353289, '2018-07-24 02:30:05'),
            (353289, '2018-07-24 03:00:15'),
            (353289, '2018-07-24 03:30:05'),
            (353289, '2018-07-24 04:00:15'),
            (353289, '2018-07-24 04:30:04'),
            ...
            (353293, '2018-07-24 00:00:14'),
            (353293, '2018-07-24 00:30:05'),
            (353293, '2018-07-24 01:00:15'),
            (353293, '2018-07-24 01:30:04'),
            (353293, '2018-07-24 02:00:14'),
            (353293, '2018-07-24 02:30:05'),
            (353293, '2018-07-24 03:00:15'),
            (353293, '2018-07-24 03:30:05'),
            (353293, '2018-07-24 04:00:15'),
            (353293, '2018-07-24 04:30:04')],
           names=['rideID', 'dateTime'], length=106)

In [33]:
input_waitTime_index[valid_targets].get_level_values(0)

UInt64Index([  353289,   353387,   353433,   353451,   353457, 17328049,
             18766536, 18774860, 18946452,   353291,
             ...
             18752875, 18752877, 18848246, 18848247, 16581435, 17328050,
             18426726, 18532806, 18693648, 18755618],
            dtype='uint64', length=106)

In [30]:
valid_targets

array([0])

In [113]:
m2 = m.values
m2[0] = np.nan

m2

array([nan, (18946452, Timestamp('2016-08-24 04:40:05'))], dtype=object)

In [136]:
df[df.status == "Operating"]["waitMins"].unique()

array([ nan,   5.,  45.,  35.,  30.,  40.,  15.,  25.,  20.,  10.,  50.,
        60.,  65.,  55.,  70., 105.,  85.,  95.,  75.,  80.,  90.,   0.,
       115., 100., 125., 120., 150., 500., 145., 140., 240., 110., 200.,
       185., 180., 130., 175., 165.,  13., 190., 160., 135., 210., 170.,
       195., 280., 205., 215., 155., 250., 900., 750.])

In [128]:
df[df.date == exdt]["status"].value_counts()

Closed                 2812
Operating              2109
Down                    137
Operates Seasonally     102
Opening Soon             32
Extra Magic Hours         2
Name: status, dtype: int64

In [104]:
df.index.get_indexer(m, method="ffill")

array([2400936])

In [91]:
sample_ts = sample_ts - timedelta(days=365)

In [59]:
ridesinfo = pd.read_csv("../disneyenv/disneyenv/envs/data/rideDuration.csv")

In [68]:
current_date = np.random.choice(avail_dates)

In [70]:
current_date.year

2018

In [51]:
rides = ridesinfo["id"].unique()

In [52]:
np.append(np.arange(len(rides)), -1)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105,  -1])

In [44]:
from scipy.spatial.distance import squareform, pdist
import geopy.distance

In [34]:
landLocation = pd.read_csv("../disneyenv/disneyenv/envs/data/landLocation.csv")

In [42]:
walking_speed = 0.0804672

In [28]:
df["dateTime"] = pd.to_datetime(df["dateTime"])

In [29]:
df["date"] = df["dateTime"].dt.date

In [53]:
avail_dates = df["date"].unique()

In [183]:
exdt = datetime.strptime("2017-09-11", "%Y-%m-%d").date()


In [184]:
df[df["date"] == exdt].waitMins.value_counts()

5.0      361
10.0     234
15.0     168
20.0      97
30.0      84
25.0      71
45.0      56
35.0      56
40.0      38
60.0      14
90.0      10
0.0       10
65.0      10
55.0       9
75.0       5
50.0       4
70.0       2
80.0       2
120.0      1
Name: waitMins, dtype: int64

In [43]:
df[(df["status"] == "Operating") & (~(df["waitMins"].isna()))]

,rideID,dateTime,waitMins,fastPassTime,status,date
22544,353291,2017-05-22 09:30:07,5.0,NaN,Operating,2017-05-22
22545,353291,2017-05-22 10:30:15,45.0,NaN,Operating,2017-05-22
22546,353291,2017-05-22 11:30:15,35.0,NaN,Operating,2017-05-22
22547,353291,2017-05-22 12:30:15,30.0,NaN,Operating,2017-05-22
22548,353291,2017-05-22 13:30:06,45.0,NaN,Operating,2017-05-22
...,...,...,...,...,...,...
2354379,18774860,2017-10-31 19:00:12,75.0,NaN,Operating,2017-10-31
2354380,18774860,2017-10-31 19:30:05,75.0,NaN,Operating,2017-10-31
2354381,18774860,2017-10-31 20:00:12,55.0,NaN,Operating,2017-10-31
2354382,18774860,2017-10-31 20:30:04,55.0,NaN,Operating,2017-10-31


In [51]:
open_df.groupby("date")["rideID"].nunique().sort_values()

date
2017-09-06    52
2017-08-30    53
2017-09-10    53
2017-09-09    53
2017-09-08    53
              ..
2017-08-11    91
2017-08-07    91
2017-08-08    91
2017-08-10    91
2017-08-06    91
Name: rideID, Length: 482, dtype: int64